# Postgres in Python - SQLAlchemy
- In this lesson, we will use the library **SQLAlchemy** to talk to a database from Python.

https://www.sqlalchemy.org/

SQL Alchemy allows us to access SQL via the Python shell/notebook!

Flask is a framework that wraps both the front-end and the back-end and allows us to work between the two!

In the front end we have our HTML, CSS, and JavaScript which interfaces with Python which in turn interacts with our back-end SQL!

### It's an Object Relational Mapper
* Map objects to relations

We're going to be mapping - creating a link - between the tables in SQL to the objects in Python

### So why do we need this?
- A layer of abstraction, to be db agnostic (T-SQL, P-SQL etc)
- Remove the need for writing SQL syntax (everyone breathes a sigh of relief!)
- Enables us to interface with other code, e.g front-end code or other Python stuff

##### *(pip install sqlalchemy)*

**Structures to understand:**

* A **Table** that represents a table in a database.
* A **class object** that represents a Table in Python.
* How **SQL Alchemy** fits into the larger picture of back and front end
* The **CRUD** statements, and their equivalent in SQL Alchemy
    * We'll learn about SQL Alchemy by doing some CRUD

### Create - a database connection
* 1: we create the necessary SQL Alchemy logistics
* 2: We create our table structures

- Setup our imports

In [1]:
from sqlalchemy import*
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Integer, String

#### Have a look at the data structure we are going to access

In [2]:
import pandas as pd
df = pd.read_csv('ml-latest-small/movies.csv')
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Start to create our ORM structures
- Create a database connection 

In [3]:
HOST = 'localhost'
DATABASE = 'movies'

mac_db_string = f'postgres://{HOST}/{DATABASE}'

### echo will print out all the SQL commands we are generating - it prints the downloading script and the SQL as well!

In [4]:
db = create_engine(mac_db_string, echo=True)

- So far, we've created a connection between Python and SQL!

### Now we can do the simplest form of SQL Alchemy interaction: 
* db.execute( )
* just put SQL syntax directly into the parantheses

In [5]:
db.execute('''INSERT INTO movies VALUES(10003, 'Star Wars II', 'Action')''')

2019-10-02 11:36:34,347 INFO sqlalchemy.engine.base.Engine select version()
2019-10-02 11:36:34,361 INFO sqlalchemy.engine.base.Engine {}
2019-10-02 11:36:34,367 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-10-02 11:36:34,367 INFO sqlalchemy.engine.base.Engine {}
2019-10-02 11:36:34,377 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-10-02 11:36:34,378 INFO sqlalchemy.engine.base.Engine {}
2019-10-02 11:36:34,396 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-10-02 11:36:34,396 INFO sqlalchemy.engine.base.Engine {}
2019-10-02 11:36:34,398 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-10-02 11:36:34,400 INFO sqlalchemy.engine.base.Engine {}
2019-10-02 11:36:34,403 INFO sqlalchemy.engine.base.Engine INSERT INTO movies VALUES(10003, 'Star Wars II', 'Action')
2019-10-02 11:36:34,404 INFO sqlalchemy.engine.base.Engine {}
2019-10-02 11:36:34,4

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "movies_pkey"
DETAIL:  Key (movieid)=(10003) already exists.
 [SQL: "INSERT INTO movies VALUES(10003, 'Star Wars II', 'Action')"] (Background on this error at: http://sqlalche.me/e/gkpj)

**But this is clunky! We can do better, using SQL Alchemy (Python) code**

---

### Now we're going to recreate CRUD statements using Python code

**Create tables**

1: Setup a declarative base allows to access all aspects of the ORM (imported this at the top)

In [6]:
base = declarative_base()

---

2: Now we create our tables:

*We use classes which is a python structure we'll cover soon*

**A CLASS IS LIKE A FUNCTION - BUT IT ALSO STORES DATA/ATTRIBUTES AS WELL AS BEHAVIOUR!**

In [7]:
class Test_Movies(base):
    
    __tablename__ = 'test_movies'
    movieId = Column(Integer, primary_key=True)
    title = Column(String)
    genres = Column(String) # Capitals because they are objects from SQL Alchemy
    
    def __repr__(self): # self tells the class it is talking to itself!
        return f'movieId = {self.movieId}, title = {self.title}'

---

3: Create all tables in the Base

This examines what we've created, and sends appropriate data to the db

In [8]:
base.metadata.create_all(db)

2019-10-02 11:37:15,647 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2019-10-02 11:37:15,648 INFO sqlalchemy.engine.base.Engine {'name': 'test_movies'}


---

#### Update - we update our tables with row information


Create a session to add items to the tables

Sessions are complicated to explain, they behave like a buffer, which remembers what we're doing in our Python code and updates the database with that information when we tell it to.

In [9]:
Session = sessionmaker(db)
session = Session()

Now add items to the tables, the workflow is:
* Create the stuff you want to add 
* Add the stuff to the session - data is not in the db (its in a buffered state), wipe the ram, wipe what you've done
* Commit the stuff to the session - data is in the db, the ram is cleared

**Lets add items one by one using add()**

In [11]:
# row1 = Test_Movies(movieid=999, title='Mission Impossible', genres='Action')
# session.add(row1)
# session.commit()

**Lets add items all together in a for loop using add_all()**

In [12]:
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
table_items= []

for i, row in df.iterrows():
    #if not session.query(Test_Movies).filter_by()
    table_items.append(Test_Movies(movieId= row[0], title=row[1], 
                                    genres=row[2]))
session.add_all(table_items)
session.commit()

2019-10-02 11:38:54,421 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-02 11:38:54,597 INFO sqlalchemy.engine.base.Engine INSERT INTO test_movies ("movieId", title, genres) VALUES (%(movieId)s, %(title)s, %(genres)s)
2019-10-02 11:38:54,598 INFO sqlalchemy.engine.base.Engine ({'movieId': 1, 'title': 'Toy Story (1995)', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy'}, {'movieId': 2, 'title': 'Jumanji (1995)', 'genres': 'Adventure|Children|Fantasy'}, {'movieId': 3, 'title': 'Grumpier Old Men (1995)', 'genres': 'Comedy|Romance'}, {'movieId': 4, 'title': 'Waiting to Exhale (1995)', 'genres': 'Comedy|Drama|Romance'}, {'movieId': 5, 'title': 'Father of the Bride Part II (1995)', 'genres': 'Comedy'}, {'movieId': 6, 'title': 'Heat (1995)', 'genres': 'Action|Crime|Thriller'}, {'movieId': 7, 'title': 'Sabrina (1995)', 'genres': 'Comedy|Romance'}, {'movieId': 8, 'title': 'Tom and Huck (1995)', 'genres': 'Adventure|Children'}  ... displaying 10 of 9742 total bound parameter s

#### Read - lets do select statements in SQL Alchemy

Return everything from a table using all()

In [15]:
# This is the same as doing SELECT * FROM test_movies;
session.query(Test_Movies).all()

2019-10-02 11:39:06,878 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-02 11:39:06,881 INFO sqlalchemy.engine.base.Engine SELECT test_movies."movieId" AS "test_movies_movieId", test_movies.title AS test_movies_title, test_movies.genres AS test_movies_genres 
FROM test_movies
2019-10-02 11:39:06,882 INFO sqlalchemy.engine.base.Engine {}


[movieId = 1, title = Toy Story (1995),
 movieId = 2, title = Jumanji (1995),
 movieId = 3, title = Grumpier Old Men (1995),
 movieId = 4, title = Waiting to Exhale (1995),
 movieId = 5, title = Father of the Bride Part II (1995),
 movieId = 6, title = Heat (1995),
 movieId = 7, title = Sabrina (1995),
 movieId = 8, title = Tom and Huck (1995),
 movieId = 9, title = Sudden Death (1995),
 movieId = 10, title = GoldenEye (1995),
 movieId = 11, title = American President, The (1995),
 movieId = 12, title = Dracula: Dead and Loving It (1995),
 movieId = 13, title = Balto (1995),
 movieId = 14, title = Nixon (1995),
 movieId = 15, title = Cutthroat Island (1995),
 movieId = 16, title = Casino (1995),
 movieId = 17, title = Sense and Sensibility (1995),
 movieId = 18, title = Four Rooms (1995),
 movieId = 19, title = Ace Ventura: When Nature Calls (1995),
 movieId = 20, title = Money Train (1995),
 movieId = 21, title = Get Shorty (1995),
 movieId = 22, title = Copycat (1995),
 movieId = 23,

#### Or make a filtered query using filter()

In [16]:
# SELECT * FROM test_movies WHERE title = 'Mission Impossible';

#session.query(Test_Movies).filter('Mission Impossible')

session.query(Test_Movies).filter(Test_Movies.title.like('Mission Impossible'))

#### Delete - let's delete some rows from our tables
- Delete items using delete( )

In [19]:
row1 = Test_Movies(movieid=999, title='Mission Impossible', genres='Action')

TypeError: 'movieid' is an invalid keyword argument for Test_Movies

In [20]:
session.delete(row1)
session.commit()

NameError: name 'row1' is not defined

In [21]:
session.query(Test_Movies).all()

2019-10-02 11:44:02,598 INFO sqlalchemy.engine.base.Engine SELECT test_movies."movieId" AS "test_movies_movieId", test_movies.title AS test_movies_title, test_movies.genres AS test_movies_genres 
FROM test_movies
2019-10-02 11:44:02,599 INFO sqlalchemy.engine.base.Engine {}


[movieId = 1, title = Toy Story (1995),
 movieId = 2, title = Jumanji (1995),
 movieId = 3, title = Grumpier Old Men (1995),
 movieId = 4, title = Waiting to Exhale (1995),
 movieId = 5, title = Father of the Bride Part II (1995),
 movieId = 6, title = Heat (1995),
 movieId = 7, title = Sabrina (1995),
 movieId = 8, title = Tom and Huck (1995),
 movieId = 9, title = Sudden Death (1995),
 movieId = 10, title = GoldenEye (1995),
 movieId = 11, title = American President, The (1995),
 movieId = 12, title = Dracula: Dead and Loving It (1995),
 movieId = 13, title = Balto (1995),
 movieId = 14, title = Nixon (1995),
 movieId = 15, title = Cutthroat Island (1995),
 movieId = 16, title = Casino (1995),
 movieId = 17, title = Sense and Sensibility (1995),
 movieId = 18, title = Four Rooms (1995),
 movieId = 19, title = Ace Ventura: When Nature Calls (1995),
 movieId = 20, title = Money Train (1995),
 movieId = 21, title = Get Shorty (1995),
 movieId = 22, title = Copycat (1995),
 movieId = 23,